In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import os
# os.chdir("..")  # Go up one level to the UROP directory
# print(os.getcwd())

/home/massimiliano/Projects/UROP


In [5]:
import torch
from utils import SensorDataset
from torch.utils.data import DataLoader
import yaml

import librosa
import matplotlib.pyplot as plt
%matplotlib inline

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

sensor = 'acc'
sr = config['sampling_rate'][sensor]

In [12]:
# Initialize Dataset
dataset = SensorDataset(root_dir="datasets/BrushlessMotor", split="train", sensors=[sensor])
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

print(f"Number of samples in dataset: {len(dataset)}")
print(dataset[0].shape)
print(dataset[5].shape)
print(dataset[6].shape)
print(dataset[7].shape)

print(f"Number of batches in dataloader: {len(dataloader)}")

Number of samples in dataset: 1281
torch.Size([37109, 3])
torch.Size([37107, 3])
torch.Size([37107, 3])
torch.Size([37107, 3])
Number of batches in dataloader: 21


In [7]:
for batch_idx, data in enumerate(dataloader):
    for sample in data:
        print(sample.shape)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/massimiliano/miniconda3/envs/urop/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/massimiliano/miniconda3/envs/urop/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/massimiliano/miniconda3/envs/urop/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/massimiliano/miniconda3/envs/urop/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 155, in collate
    return collate_fn_map[elem_type](batch, col

In [ ]:
random_index = torch.randint(0, 64, (1,)).item()
mel_spectrogram = librosa.feature.melspectrogram(y=data[random_index, :, 0].numpy(), sr=sr)
mel_log_spectrogram = librosa.power_to_db(mel_spectrogram)

plt.figure()
librosa.display.specshow(mel_log_spectrogram, x_axis='time', y_axis='mel', sr=sr, cmap='plasma')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.show()